In [2]:
import torch
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv("./data/uid_click_log.csv")

In [5]:
data

,uid,vid,cid,playtime,duration,date,rank
0,100000,100870,100037,4966,9590,20200706,1
1,100000,101167,100024,5037,4949,20200706,2
2,100000,103608,100008,5137,14884,20200706,3
3,100000,100220,100084,5139,16784,20200706,4
4,100000,101674,100027,5149,5760,20200706,5
...,...,...,...,...,...,...,...
3487097,105999,102918,100019,124488,116467,20200810,1
3487098,105999,100917,100019,88168,120500,20200827,1
3487099,105999,100208,100079,49005,57506,20200828,1
3487100,105999,106468,100027,26886,8067,20200904,1
